In [1]:
from bs4 import BeautifulSoup
import json 
import numpy as np
import pandas as pd
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
import urllib
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

In [2]:
pd.options.display.max_rows = 4000

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
response = urllib.request.urlopen(url)
rawhtml = response.read()

In [5]:
soup = BeautifulSoup(rawhtml, 'html.parser')

In [6]:
pclist = []
for table_row in soup.select("table.wikitable tr"):
    cells = table_row.findAll('td')
    if len(cells) > 0:
        pc = cells[0].text.strip()
        b = cells[1].text.strip()
        n = cells[2].text.strip()
        if (b == "Not assigned" and n=="Not assigned"):
            a="skipping"
        elif(n=="Not assigned"):
            pclist.append([pc,b,b])
        else:
            pclist.append([pc,b,n])

In [7]:
toronto_df = pd.DataFrame(pclist, columns=["FSA", "Borough", "Neighborhood"])
toronto_df = toronto_df.sort_values(by=['FSA'])
toronto_df.head(15)


,FSA,Borough,Neighborhood
0,M1A,Not assigned,
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae
54,M1J,Scarborough,Scarborough Village
63,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
72,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
81,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


In [8]:
pclist = []
prevPC = "" 
currNeighbourhoods = ""
prevBorough = ""
for table_row in soup.select("table.wikitable tr"):
    cells = table_row.findAll('td')
    if len(cells) > 0:
        pc = cells[0].text.strip()
        b = cells[1].text.strip()
        n = cells[2].text.strip()
        if ((b == "Not assigned" and n=="Not assigned") or b == "blank"):
            a="skipping"
        else:
            if (n=="Not assigned"):
                n = b; 
            if (pc == prevPC):
                currNeighbourhoods = currNeighbourhoods + ", " + n 
            else:
                if (prevPC != ""):
                    pclist.append([prevPC,prevBorough,currNeighbourhoods])
                prevPC = pc 
                prevBorough = b
                currNeighbourhoods = n 
pclist.append([prevPC,prevBorough,currNeighbourhoods])                 
toronto_df = pd.DataFrame(pclist, columns=["PostalCode", "Borough", "Neighborhood"])
toronto_df = toronto_df.sort_values(by=['PostalCode'])
toronto_df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae
54,M1J,Scarborough,Scarborough Village
63,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
72,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
81,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
